In [1]:
import pandas as pd
import scipy as sc
import numpy as np
import sklearn
import pickle
import pathlib as Path
import matplotlib.pyplot as plt
import japanize_matplotlib
import seaborn as sb
sb.set(font='IPAexGothic')

import multiprocessing
import itertools
import collections
import datetime
import gc

from tqdm._tqdm_notebook import tqdm

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.set_option('display.max_columns', 50)

data_path = Path.Path("../data")
result_path = Path.Path("../result")

prefix = 'ana403'

# データ抽出
> ・ヒット確率付与済みデータ（ana301_all_FTR_prob_df.pkl）

In [2]:
all_df = pd.read_pickle(data_path / "ana301_all_FTR_prob_df.pkl")

In [3]:
all_df

,id,gameID,inning,batter,pitcher,y,is_hit0,is_hit1,is_hit2,is_hit3,is_hit4,flg_train,FTR_cnt_B,FTR_cnt_O,FTR_cnt_S,FTR_pting_cnt,FTR_inning,FTR_bt_btingavg00,FTR_bt_btingavg01,FTR_bt_btingavg02,FTR_bt_btingavg03,FTR_bt_btingavg04,FTR_bt_btingavg10,FTR_bt_btingavg11,FTR_bt_btingavg12,...,FTR_pt_btingavg11,FTR_pt_btingavg12,FTR_pt_btingavg13,FTR_pt_btingavg14,FTR_pt_btingavg20,FTR_pt_btingavg21,FTR_pt_btingavg22,FTR_pt_btingavg23,FTR_pt_btingavg24,FTR_pt_btingavg30,FTR_pt_btingavg31,FTR_pt_btingavg32,FTR_pt_btingavg33,FTR_pt_btingavg34,FTR_pt_btingavg40,FTR_pt_btingavg41,FTR_pt_btingavg42,FTR_pt_btingavg43,FTR_pt_btingavg44,FTR_pt_btingavg50,FTR_pt_btingavg51,FTR_pt_btingavg52,FTR_pt_btingavg53,FTR_pt_btingavg54,FTR_prob_logit
0,0,20202173,1回表,ピレラ,今永 昇太,0.0,False,False,False,False,False,1,0,0,0,1,1,0.269122,0.184136,0.036827,0.005666,0.042493,0.333333,0.238095,0.047619,...,0.461538,0.076923,0.153846,0.000000,0.538462,0.461538,0.000000,0.076923,0.000000,0.800000,0.600000,0.200000,0.4,0.000000,0.538462,0.384615,0.000000,0.153846,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.197120
1,1,20202173,1回表,ピレラ,今永 昇太,1.0,False,False,False,False,False,1,1,0,0,2,1,0.269122,0.184136,0.036827,0.005666,0.042493,0.333333,0.238095,0.047619,...,0.461538,0.076923,0.153846,0.000000,0.538462,0.461538,0.000000,0.076923,0.000000,0.800000,0.600000,0.200000,0.4,0.000000,0.538462,0.384615,0.000000,0.153846,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.076125
2,2,20202173,1回表,ピレラ,今永 昇太,0.0,False,False,False,False,False,1,1,0,1,3,1,0.269122,0.184136,0.036827,0.005666,0.042493,0.333333,0.238095,0.047619,...,0.461538,0.076923,0.153846,0.000000,0.538462,0.461538,0.000000,0.076923,0.000000,0.800000,0.600000,0.200000,0.4,0.000000,0.538462,0.384615,0.000000,0.153846,0.000000,0.111111,0.111111,0.000000,0.0,0.000000,0.066053
3,3,20202173,1回表,ピレラ,今永 昇太,2.0,False,False,False,False,False,1,2,0,1,4,1,0.269122,0.184136,0.036827,0.005666,0.042493,0.333333,0.238095,0.047619,...,0.461538,0.076923,0.153846,0.000000,0.538462,0.461538,0.000000,0.076923,0.000000,0.800000,0.600000,0.200000,0.4,0.000000,0.538462,0.384615,0.000000,0.153846,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.008747
4,4,20202173,1回表,ピレラ,今永 昇太,4.0,True,True,False,False,False,1,2,0,2,5,1,0.269122,0.184136,0.036827,0.005666,0.042493,0.333333,0.238095,0.047619,...,0.461538,0.076923,0.153846,0.000000,0.538462,0.461538,0.000000,0.076923,0.000000,0.800000,0.600000,0.200000,0.4,0.000000,0.538462,0.384615,0.000000,0.153846,0.000000,0.500000,0.500000,0.000000,0.0,0.000000,0.926282
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54203,33803,20202023,5回表,坂口 智隆,床田 寛樹,NaN,False,False,False,False,False,0,0,0,1,2,5,0.226006,0.154799,0.043344,0.000000,0.030960,0.250000,0.200000,0.025000,...,0.714286,0.142857,0.000000,0.142857,0.666667,0.666667,0.000000,0.000000,0.000000,0.714286,0.714286,0.142857,0.0,0.142857,0.714286,0.571429,0.142857,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.115175
54204,33804,20202640,9回表,メヒア,堀岡 隼人,NaN,False,False,False,False,False,0,0,0,0,1,9,0.269122,0.184136,0.036827,0.005666,0.042493,0.121212,0.090909,0.000000,...,0.404040,0.121212,0.000000,0.111111,0.505051,0.404040,0.121212,0.000000,0.111111,0.505051,0.404040,0.121212,0.0,0.111111,0.505051,0.404040,0.121212,0.000000,0.111111,0.505051,0.404040,0.121212,0.0,0.111111,0.783062
54205,33805,20202864,7回裏,鈴木 誠也,ディプラン,NaN,False,False,False,False,False,0,0,0,0,1,7,0.269122,0.184136,0.036827,0.005666,0.042493,0.325000,0.150000,0.025000,...,0.404040,0.121212,0.000000,0.111111,0.505051,0.404040,0.121212,0.000000,0.111111,0.505051,0.404040,0.121212,0.0,0.111111,0.505051,0.404040,0.121212,0.000000,0.111111,0.505051,0.404040,0.121212,0.0,0.111111,0.947061
54206,33806,20202806,8回裏,周東 佑京,田村 伊知郎,NaN,False,False,False,False,False,0,3,1,1,5,8,0.190981,0.119363,0.037135,0.005305

# 訓練データ抽出

In [4]:
train_df = all_df[all_df['flg_train']==1]
train_df.shape

(20400, 78)

# 　

# モデル構築
> 目的変数：y <br>
> 説明変数：FTR_: <br>
> モデル：多値分類 <br>
> 　　・k-means <br>
> チューニング：CV 

## 下準備

In [5]:
# 目的変数
y = train_df['y']

# 説明変数
X = train_df.filter(like='FTR', axis=1)

In [6]:
# K-Means
from sklearn.cluster import KMeans

## モデル推定
> KMeans( <br>
> 　　　　algorithm='auto', copy_x=True, init='k-means++', max_iter=300, <br>
> 　　　　n_clusters=8, n_init=10, n_jobs=None, precompute_distances='auto', <br>
> 　　　　random_state=None, tol=0.0001, verbose=0 <br>
> ) <br>

In [7]:
# モデル推定
model = KMeans(n_clusters=8, random_state=123, max_iter=10000).fit(X, y)

In [8]:
# model.best_estimator_.score(X, y)
model.score(X, y)

-74661.17064516262

## モデル予測

In [9]:
# 全体データ
pred = model.predict(
           
            # 全体データの特徴量
            all_df.filter(like="FTR_", axis=1)
        )

# 　

# データ結合
> ・id, flg_train + 予測値（pred）

In [10]:
# id, データ分割情報
pred_df = all_df[['id', 'flg_train', 'y']].copy()

# 予測値
pred_df['pred'] = pred

In [11]:
pred_df

,id,flg_train,y,pred
0,0,1,0.0,5
1,1,1,1.0,5
2,2,1,0.0,3
3,3,1,2.0,3
4,4,1,4.0,3
...,...,...,...,...
54203,33803,0,NaN,1
54204,33804,0,NaN,6
54205,33805,0,NaN,6
54206,33806,0,NaN,4


In [12]:
pred_df[pred_df['flg_train']==1].groupby(['y', 'pred']).count()

id  flg_train
y   pred                 
0.0 0      391        391
    1     1384       1384
    2     1050       1050
    3      724        724
    4      461        461
...        ...        ...
7.0 3       24         24
    4       16         16
    5       24         24
    6       17         17
    7       11         11

[61 rows x 2 columns]

In [13]:
pred_df.groupby('pred').count()

,id,flg_train,y
pred,,,
0,3059,3059,1497
1,10037,10037,3372
2,7283,7283,2916
3,4868,4868,2000
4,3445,3445,1593
5,9961,9961,3369
6,9093,9093,3074
7,6462,6462,2579


# データ保存

In [14]:
pred_df.to_pickle(data_path / ("%s_pred_kmeans_df.pkl" % prefix))